### Imports

In [1]:
import json
import random
import re

import nltk
from nltk import CFG
from nltk.parse.generate import generate

random.seed(123)

### Items Dictionary from Items Intent
Including
* item name
* slot name
* location details
* neighboring items
* content items (optional)
* adjectives
* hyperonyms

In [2]:
items_dict = {
    "astronautinnen": {
        "slot": "astronautinnen",
        "location": ["links", "oben", "in der ecke"],
        "neighbors": ["ufo", "raumschiff"],
        "content": ["anzügen"],
        "adjectives": ["3", "weiß"],
		"hyperonyms": ["menschen", "personen", "frauen"],
    },
    "ballon": {
        "slot": "ballon",
        "location": [
            "rechts",
            "unten",
            "in der ecke",
            "am rand",
            "auf der rechten seite",
            "über",
        ],
        "neighbors": ["boot", "schiff"],
        "content": [],
        "adjectives": ["rot", "rund"],
		"hyperonyms": ["ding", "gegenstand", "objekt", "spielzeug"],
    },
    "biene": {
        "slot": "biene",
        "location": ["rechts", "am rand", "auf der rechten seite"],
        "neighbors": [],
        "content": ["streifen"],
        "adjectives": ["gelb", "gestreift"],
		"hyperonyms": ["tier", "insekt", "lebewesen"],
    },
    "boot": {
        "slot": "boot",
        "location": [
            "rechts",
            "unten",
            "in der ecke",
            "am rand",
            "auf der rechten seite",
            "unter",
            "auf",
        ],
        "neighbors": ["ballon", "wasser", "fluss", "roboter"],
        "content": [],
        "adjectives": ["grün", "lächelnd", "schwimmend"],
		"hyperonyms": ["ding", "gegenstand", "objekt", "fahrzeug"],
    },
    "buchLinksOben": {
        "slot": "buch",
        "location": ["links", "oben", "in der ecke"],
        "neighbors": ["buch", "uhr", "säulen"],
        "content": [],
        "adjectives": ["grün", "blau", "geschlossen"],
		"hyperonyms": ["ding", "gegenstand", "objekt"],
    },
    "buchMitteUnten": {
        "slot": "buch",
        "location": ["in der mitte", "unten", "am rand"],
        "neighbors": ["pfeilen", "tisch", "kurven"],
        "content": [],
        "adjectives": ["grün", "geschlossen", "klein"],
		"hyperonyms": ["ding", "gegenstand", "objekt"],
    },
    "buchLinksObenOffen": {
        "slot": "buch",
        "location": ["links", "oben", "in der ecke"],
        "neighbors": ["buch", "uhr", "säulen"],
        "content": [],
        "adjectives": ["aufgeschlagen", "offen", "geöffnet", "klein"],
		"hyperonyms": ["ding", "gegenstand", "objekt"],
    },
    "buchRechtsMitte": {
        "slot": "buch",
        "location": ["rechts", "auf der rechten seite", "unter"],
        "neighbors": ["globus", "spinne", "rauch", "mikroskop"],
        "content": [],
        "adjectives": ["aufgeschlagen", "offen", "geöffnet"],
		"hyperonyms": ["ding", "gegenstand", "objekt"],
    },
    "diagramm": {
        "slot": "diagramm",
        "location": ["rechts", "oben", "am rand", "über", "neben", "an der wand"],
        "neighbors": ["mikroskop", "uhr"],
        "content": ["x", "y", "sigma", "summe", "summenzeichen", "normalverteilung", "gaußkurve", "mathe", "mathematik", "achse"],
        "adjectives": ["bunt", "schwarz", "mathematisch"],
		"hyperonyms": ["ding", "gegenstand", "objekt"],
    },
    "europa": {
        "slot": "europa",
        "location": [
            "unten",
            "links",
            "auf der linken seite",
            "über",
            "unter",
            "neben",
        ],
        "neighbors": ["tisch", "schultasche"],
        "content": ["sterne", "sternen"],
        "adjectives": ["blau", "gelb"],
		"hyperonyms": ["ding", "gegenstand", "objekt"],
    },
    "fff": {
        "slot": "kinder",
        "location": ["oben", "links", "hinter", "unter"],
        "neighbors": ["terminal", "konsole", "astronautinnen", "wolke"],
        "content": ["fahne"],
        "adjectives": ["grün", "beid"],
		"hyperonyms": ["menschen", "personen", "kinder"],
    },
    "gleichung": {
        "slot": "gleichung",
        "location": ["unten", "rechts"],
        "neighbors": ["stift", "kolben", "chemie", "labor"],
        "content": ["x", "y", "Gleichheitszeichen"],
        "adjectives": ["weiß"],
		"hyperonyms": ["ding", "gegenstand", "objekt"],
    },
    "globus": {
        "slot": "globus",
        "location": ["in der mitte", "im zentrum"],
        "neighbors": ["wolke", "treppe"],
        "content": ["sockel"],
        "adjectives": ["mittig", "zentral", "rund", "blau", "grün"],
		"hyperonyms": ["ding", "gegenstand", "objekt"],
    },
    "gluehbirneObenMitteGross": {
        "slot": "glühbirne",
        "location": ["oben", "im oberen teil", "in der mitte", "neben"],
        "neighbors": ["glühbirne", "teleskop"],
        "content": [],
        "adjectives": ["gelb", "groß", "rund"],
		"hyperonyms": ["ding", "gegenstand", "objekt"],
    },
    "gluehbirneObenMitteKlein": {
        "slot": "glühbirne",
        "location": ["oben", "im oberen teil", "in der mitte", "neben"],
        "neighbors": ["glühbirne", "globus"],
        "content": [],
        "adjectives": ["gelb", "klein", "rund"],
		"hyperonyms": ["ding", "gegenstand", "objekt"],
    },
    "kurven": {
        "slot": "kurven",
        "location": ["unten", "links", "mittig", "links von der mitte", "in der mitte"],
        "neighbors": ["buch", "lineal", "tafel", "wolken"],
        "content": [],
        "adjectives": ["weiß", "bunt", "gelb", "rosa", "pink"],
		"hyperonyms": ["ding", "gegenstand", "objekt"],
    },
    "lab": {
        "slot": "labor",
        "location": ["unten", "rechts", "am rand", "neben"],
        "neighbors": ["wasser", "boot", "ballon"],
        "content": ["terminal", "diy", "kasten"],
        "adjectives": ["eckig", "bunt", "rot"],
		"hyperonyms": ["ding", "gegenstand", "objekt"],
    },
    "mikroskop": {
        "slot": "mikroskop",
        "location": ["oben", "rechts", "neben", "unter"],
        "neighbors": ["diagramm", "buch", "tisch", "pflanze", "kugeln"],
        "content": [],
        "adjectives": ["grau", "grün", "petrol"],
		"hyperonyms": ["ding", "gegenstand", "objekt"],
    },
    "personLinksMitte": {
        "slot": "person",
        "location": ["links", "in der mitte", "neben"],
        "neighbors": ["taube", "vogel", "wolke", "stift"],
        "content": [],
        "adjectives": ["sitzend", "kauernd", "blond", "lila"],
		"hyperonyms": ["mensch", "person"],
    },
    "personMitteMitte": {
        "slot": "person",
        "location": ["in der mitte", "im zentrum", "neben"],
        "neighbors": ["terminal", "konsole", "ordner", "kästen"],
        "content": ["kittel", "brille", "klemmbrett"],
        "adjectives": ["weiß", "grau"],
		"hyperonyms": ["mensch", "person"],
    },
    "personRechtsMitte": {
        "slot": "person",
        "location": ["in der mitte", "rechts", "neben"],
        "neighbors": ["labor", "magnet", "kolben", "sprechblase"],
        "content": ["kittel", "brille"],
        "adjectives": ["weiß", "blond", "gelb"],
		"hyperonyms": ["mensch", "person"],
    },
    "planet": {
        "slot": "planet",
        "location": [
            "oben",
            "links",
            "auf der linken seite",
            "am Rand",
            "neben",
        ],
        "neighbors": ["buch", "astronautinnen"],
        "content": [],
        "adjectives": ["gelb", "gold", "rund"],
		"hyperonyms": ["ding", "gegenstand", "objekt"],
    },
    "raumschiff": {
        "slot": "raumschiff",
        "location": [
            "oben",
            "links",
            "auf der linken seite",
            "am Rand",
            "neben",
        ],
        "neighbors": ["astronautinnen", "teleskop", "glühbirne", "terminal"],
        "content": [],
        "adjectives": ["rot", "eckig"],
		"hyperonyms": ["ding", "gegenstand", "objekt", "fahrzeug", "flugobjekt"],
    },
    "spielKonsole": {
        "slot": "spielkonsole",
        "location": ["in der mitte", "im zentrum", "neben"],
        "neighbors": ["glühbirne", "terminal", "astronautinnen", "würfel"],
        "content": ["tasten", "knöpfen"],
        "adjectives": ["eckig", "schwarz", "grau", "flach"],
		"hyperonyms": ["ding", "gegenstand", "objekt"],
    },
    "stiftLinksMitte": {
        "slot": "stift",
        "location": [
            "mittig",
            "links",
            "auf der linken seite",
            "am Rand",
            "neben",
        ],
        "neighbors": ["person", "würfel", "tafel"],
        "content": [],
        "adjectives": ["grün", "spitz", "länglich"],
		"hyperonyms": ["ding", "gegenstand", "objekt", "schreibutensil"],
    },
    "stiftRechtsUnten": {
        "slot": "stift",
        "location": [
            "unten",
            "rechts",
            "auf der rechten seite",
            "am Rand",
            "neben",
        ],
        "neighbors": ["labor", "ballon", "boot", "gleichung", "konsole", "terminal"],
        "content": [],
        "adjectives": ["rot", "spitz", "länglich"],
		"hyperonyms": ["ding", "gegenstand", "objekt", "schreibutensil"],
    },
    "tafel": {
        "slot": "tafel",
        "location": [
            "unten",
            "links",
            "auf der linken seite",
            "am Rand",
            "neben",
            "bei",
        ],
        "neighbors": ["würfel", "stift", "buch", "kurven", "europa"],
        "content": ["zwei", "vier", "komma", "punkt"],
        "adjectives": ["grün", "beschriftet", "beschrieben"],
		"hyperonyms": ["ding", "gegenstand", "objekt"],
    },
    "teleskop": {
        "slot": "teleskop",
        "location": ["am Rand", "mittig", "neben", "neben"],
        "neighbors": ["glühbirne", "raumschiff"],
        "content": [],
        "adjectives": ["rot", "rund", "mikroskopähnlich"],
		"hyperonyms": ["ding", "gegenstand", "objekt"],
    },
    "terminalMitteMitte": {
        "slot": "terminal",
        "location": [
            "in der mitte",
            "im zentrum",
            "neben",
        ],
        "neighbors": [
            "globus",
            "spielkonsole",
            "glühbirne",
            "astronautinnen",
            "würfel",  
        ],
        "content": ["schrift", "hello", "world"],
        "adjectives": ["schwarz", "beschriftet"],
		"hyperonyms": ["ding", "gegenstand", "objekt"],
    },
    "uhrLinksOben": {
        "slot": "uhr",
        "location": ["oben", "links", "auf der linken seite", "neben"],
        "neighbors": ["buch", "astronautinnen", "person", "planet",],
        "content": ["zeit", "zeigern"],
        "adjectives": ["gelb", "rund"],
		"hyperonyms": ["ding", "gegenstand", "objekt"],
    },
    "uhrRechtsOben": {
        "slot": "uhr",
        "location": [
            "oben",
            "rechts",
            "auf der rechten seite",
            "am Rand",
            "an der Wand",
            "neben",
        ],
        "neighbors": ["diagramm", "mikroskop"],
        "content": ["zeit", "zeigern"],
        "adjectives": ["blau", "rund", "grün"],
		"hyperonyms": ["ding", "gegenstand", "objekt"],
    },
    "wuerfel": {
        "slot": "würfel",
        "location": [
            "mittig",
            "links",
            "auf der linken seite",
            "neben",
        ],
        "neighbors": ["person", "spielkonsole", "kurven", "terminal"],
        "content": ["buchstaben", "a", "b"],
        "adjectives": ["blau", "türkis", "gelb", "lila", "bunt"],
		"hyperonyms": ["ding", "gegenstand", "objekt"],
    },
}

### Interaction Model with detailed slot names, values, and synonyms

In [3]:
model = json.load(open('interaction_model.json', 'r', encoding='utf-8'))

In [4]:
# create a nested dictionary from interaction model slots {slot: dict of slot values: list of their synonyms}
slots = dict()
for s in model["interactionModel"]["languageModel"]["types"]:
    slots[s['name']] = dict()
    for v in s['values']:
        slots[s['name']][v['name']['value']] = list()
        if 'synonyms' in v['name']:
            slots[s['name']][v['name']['value']] = v['name']['synonyms']
        #slots[s['name']] += [v['name']['value']]
    # replace ' by `
    #for slot, index in zip(slots[s['name']], range(len(slots[s['name']]))):
        #slots[s['name']][index] = slot.replace('\'', '`')
print(slots['object'])

{'other': ['Absolvent', 'Student', 'Gesicht', 'Smiley', 'Sterne', 'Stern', 'Bildschirm', 'Podest', 'Binärcode', 'Stühle', 'Stuhl', 'Seeigel', 'Mine', 'Pfeile', 'Pfeil', 'Legobaustein', 'Legostein', 'Tisch', 'ABC', 'Molekül', 'Atom', 'Schrank', 'Delfin', 'Medaille', 'Tauben', 'Taube', 'Vögel', 'Vogel', 'Pfeile', 'Pfeil', 'Regenbogen', 'Reagenzglas', 'Schultasche', 'Säulen', 'Spinne', 'Wolken', 'Wolke', 'Blumentopf', 'Pflanze', 'Treppe', 'Lineal', 'Büro', 'Schreibtisch', 'Giraffenkopf', 'Giraffe', 'Zahnrad'], 'Ding': ['Sachen', 'Sache', 'Teile', 'Gegenstände', 'Dinger', 'Dingens', 'Dings', 'Objekt', 'Gegenstand', 'Teil'], 'Uhr': [], 'Würfel': ['Buchstabenwürfel'], 'Terminal': ['Programmfenster', 'Bildschirm', 'Computerbildschirm', 'Konsole'], 'Teleskop': ['Fernrohr'], 'Tafel': ['Schultafel'], 'Stift': ['Schreibutensil', 'Buntstift'], 'Spielkonsole': ['Gameboy', 'Switch', 'Spielekonsole'], 'Raumschiff': ['Spaceshuttle', 'Flugzeug'], 'Planet': ['Saturn'], 'Person': ['Ärztin', 'Arzt', 'Inge

### Encode grammatical properties of nouns, determiners, and verbs/sentence beginnings (BOS)

In [5]:
OBJECTS = """
N[CAS=?c,NUM=sg,GEN=m] -> 'Absolvent'
N[CAS=?c,NUM=sg,GEN=m] -> 'Student'
N[CAS=?c,NUM=sg,GEN=n] -> 'Gesicht'
N[CAS=?c,NUM=sg,GEN=m] -> 'Smiley'
N[CAS=?c,NUM=pl,GEN=m] -> 'Sterne'
N[CAS=dat,NUM=pl,GEN=m] -> 'Sternen'
N[CAS=?c,NUM=sg,GEN=m] -> 'Stern'
N[CAS=?c,NUM=sg,GEN=m] -> 'Bildschirm'
N[CAS=?c,NUM=sg,GEN=n] -> 'Podest'
N[CAS=?c,NUM=sg,GEN=m] -> 'Binärcode'
N[CAS=?c,NUM=pl,GEN=m] -> 'Stühle'
N[CAS=?c,NUM=sg,GEN=m] -> 'Stuhl'
N[CAS=?c,NUM=sg,GEN=m] -> 'Seeigel'
N[CAS=?c,NUM=sg,GEN=f] -> 'Mine'
N[CAS=?c,NUM=pl,GEN=m] -> 'Pfeile'
N[CAS=dat,NUM=pl,GEN=m] -> 'Pfeilen'
N[CAS=?c,NUM=sg,GEN=m] -> 'Pfeil'
N[CAS=?c,NUM=sg,GEN=m] -> 'Legobaustein'
N[CAS=?c,NUM=sg,GEN=m] -> 'Legostein'
N[CAS=?c,NUM=sg,GEN=m] -> 'Tisch'
N[CAS=?c,NUM=sg,GEN=n] -> 'ABC'
N[CAS=?c,NUM=sg,GEN=n] -> 'Molekül'
N[CAS=?c,NUM=sg,GEN=n] -> 'Atom'
N[CAS=?c,NUM=sg,GEN=m] -> 'Schrank'
N[CAS=?c,NUM=sg,GEN=m] -> 'Delfin'
N[CAS=?c,NUM=sg,GEN=f] -> 'Medaille'
N[CAS=?c,NUM=pl,GEN=f] -> 'Tauben'
N[CAS=?c,NUM=sg,GEN=f] -> 'Taube'
N[CAS=?c,NUM=pl,GEN=m] -> 'Vögel'
N[CAS=?c,NUM=sg,GEN=m] -> 'Vogel'
N[CAS=?c,NUM=pl,GEN=m] -> 'Pfeile'
N[CAS=?c,NUM=sg,GEN=m] -> 'Pfeil'
N[CAS=?c,NUM=sg,GEN=m] -> 'Regenbogen'
N[CAS=?c,NUM=sg,GEN=n] -> 'Reagenzglas'
N[CAS=?c,NUM=sg,GEN=f] -> 'Schultasche'
N[CAS=?c,NUM=pl,GEN=f] -> 'Säulen'
N[CAS=?c,NUM=sg,GEN=f] -> 'Spinne'
N[CAS=?c,NUM=pl,GEN=f] -> 'Wolken'
N[CAS=?c,NUM=sg,GEN=f] -> 'Wolke'
N[CAS=?c,NUM=sg,GEN=m] -> 'Blumentopf'
N[CAS=?c,NUM=sg,GEN=f] -> 'Pflanze'
N[CAS=?c,NUM=sg,GEN=f] -> 'Treppe'
N[CAS=?c,NUM=sg,GEN=n] -> 'Lineal'
N[CAS=?c,NUM=sg,GEN=n] -> 'Büro'
N[CAS=?c,NUM=sg,GEN=m] -> 'Schreibtisch'
N[CAS=?c,NUM=sg,GEN=m] -> 'Giraffenkopf'
N[CAS=?c,NUM=sg,GEN=f] -> 'Giraffe'
N[CAS=?c,NUM=sg,GEN=n] -> 'Zahnrad'
N[CAS=?c,NUM=sg,GEN=?g] -> 'other'
N[CAS=?c,NUM=pl,GEN=f] -> 'Sachen'
N[CAS=?c,NUM=sg,GEN=f] -> 'Sache'
N[CAS=?c,NUM=pl,GEN=n] -> 'Teile'
N[CAS=?c,NUM=pl,GEN=m] -> 'Gegenstände'
N[CAS=?c,NUM=pl,GEN=n] -> 'Dinger'
N[CAS=?c,NUM=sg,GEN=n] -> 'Dingens'
N[CAS=?c,NUM=sg,GEN=n] -> 'Dings'
N[CAS=?c,NUM=sg,GEN=n] -> 'Objekt'
N[CAS=?c,NUM=sg,GEN=m] -> 'Gegenstand'
N[CAS=?c,NUM=sg,GEN=n] -> 'Teil'
N[CAS=?c,NUM=sg,GEN=n] -> 'Ding'
N[CAS=?c,NUM=sg,GEN=f] -> 'Uhr'
N[CAS=?c,NUM=sg,GEN=m] -> 'Buchstabenwürfel'
N[CAS=?c,NUM=sg,GEN=m] -> 'Würfel'
N[CAS=?c,NUM=sg,GEN=n] -> 'Programmfenster'
N[CAS=?c,NUM=sg,GEN=m] -> 'Bildschirm'
N[CAS=?c,NUM=sg,GEN=m] -> 'Computerbildschirm'
N[CAS=?c,NUM=sg,GEN=f] -> 'Konsole'
N[CAS=?c,NUM=sg,GEN=m] -> 'Terminal'
N[CAS=?c,NUM=sg,GEN=n] -> 'Fernrohr'
N[CAS=?c,NUM=sg,GEN=n] -> 'Teleskop'
N[CAS=?c,NUM=sg,GEN=f] -> 'Schultafel'
N[CAS=?c,NUM=sg,GEN=f] -> 'Tafel'
N[CAS=?c,NUM=sg,GEN=n] -> 'Schreibutensil'
N[CAS=?c,NUM=sg,GEN=m] -> 'Buntstift'
N[CAS=?c,NUM=sg,GEN=m] -> 'Stift'
N[CAS=?c,NUM=sg,GEN=m] -> 'Gameboy'
N[CAS=?c,NUM=sg,GEN=f] -> 'Switch'
N[CAS=?c,NUM=sg,GEN=f] -> 'Spielekonsole'
N[CAS=?c,NUM=sg,GEN=f] -> 'Spielkonsole'
N[CAS=?c,NUM=sg,GEN=n] -> 'Spaceshuttle'
N[CAS=?c,NUM=sg,GEN=n] -> 'Flugzeug'
N[CAS=?c,NUM=sg,GEN=n] -> 'Raumschiff'
N[CAS=?c,NUM=sg,GEN=m] -> 'Saturn'
N[CAS=?c,NUM=sg,GEN=m] -> 'Planet'
N[CAS=?c,NUM=sg,GEN=f] -> 'Ärztin'
N[CAS=?c,NUM=sg,GEN=m] -> 'Arzt'
N[CAS=?c,NUM=sg,GEN=f] -> 'Ingenieurin'
N[CAS=?c,NUM=sg,GEN=m] -> 'Ingenieur'
N[CAS=?c,NUM=sg,GEN=m] -> 'Mann'
N[CAS=?c,NUM=sg,GEN=f] -> 'Frau'
N[CAS=?c,NUM=sg,GEN=f] -> 'Wissenschaftlerin'
N[CAS=?c,NUM=sg,GEN=m] -> 'Wissenschaftler'
N[CAS=?c,NUM=sg,GEN=f] -> 'Forscherin'
N[CAS=?c,NUM=sg,GEN=m] -> 'Forscher'
N[CAS=?c,NUM=pl,GEN=f] -> 'Personen'
N[CAS=?c,NUM=sg,GEN=m] -> 'Mensch'
N[CAS=?c,NUM=sg,GEN=f] -> 'Person'
N[CAS=?c,NUM=sg,GEN=n] -> 'Teleskop'
N[CAS=?c,NUM=sg,GEN=n] -> 'Mikroskop'
N[CAS=?c,NUM=sg,GEN=m] -> 'Kasten'
N[CAS=?c,NUM=sg,GEN=n] -> 'Lab'
N[CAS=?c,NUM=sg,GEN=n] -> 'Labor'
N[CAS=?c,NUM=pl,GEN=f] -> 'Grafiken'
N[CAS=?c,NUM=sg,GEN=n] -> 'Diagramm'
N[CAS=?c,NUM=pl,GEN=f] -> 'Wellen'
N[CAS=?c,NUM=pl,GEN=f] -> 'Kurven'
N[CAS=?c,NUM=sg,GEN=f] -> 'Birne'
N[CAS=?c,NUM=sg,GEN=f] -> 'Glühbirne'
N[CAS=?c,NUM=sg,GEN=m] -> 'Screen'
N[CAS=?c,NUM=sg,GEN=f] -> 'Gleichung'
N[CAS=?c,NUM=sg,GEN=f] -> 'Demo'
N[CAS=?c,NUM=pl,GEN=f] -> 'Klimaaktivistinnen'
N[CAS=?c,NUM=pl,GEN=m] -> 'Klimaaktivisten'
N[CAS=?c,NUM=?n,GEN=?g] -> 'Fridays for Future'
N[CAS=?c,NUM=?n,GEN=?g] -> 'FFF'
N[CAS=?c,NUM=pl,GEN=n] -> 'Kinder mit Flagge'
N[CAS=?c,NUM=pl,GEN=?g] -> 'Friends For Future'
N[CAS=?c,NUM=pl,GEN=?g] -> 'Friends'
N[CAS=?c,NUM=sg,GEN=?g] -> 'Future'
N[CAS=?c,NUM=pl,GEN=n] -> 'Kinder'
N[CAS=?c,NUM=sg,GEN=f] -> 'Flagge'
N[CAS=?c,NUM=sg,GEN=f] -> 'Fahne'
N[CAS=?c,NUM=sg,GEN=f] -> 'EU'
N[CAS=?c,NUM=sg,GEN=f] -> 'Europaflagge'
N[CAS=?c,NUM=sg,GEN=n] -> 'Europa'
N[CAS=?c,NUM=sg,GEN=f] -> 'Tafel'
N[CAS=?c,NUM=sg,GEN=n] -> 'Koordinatensystem'
N[CAS=?c,NUM=sg,GEN=n] -> 'Diagramm'
N[CAS=?c,NUM=sg,GEN=n] -> 'Heft'
N[CAS=?c,NUM=sg,GEN=n] -> 'Buch'
N[CAS=?c,NUM=sg,GEN=n] -> 'Schiff'
N[CAS=?c,NUM=sg,GEN=n] -> 'Boot'
N[CAS=?c,NUM=pl,GEN=m] -> 'Drillinge'
N[CAS=?c,NUM=pl,GEN=f] -> 'Raumfahrerinnen'
N[CAS=?c,NUM=pl,GEN=f] -> 'Frauen'
N[CAS=?c,NUM=pl,GEN=f] -> 'Astronautinnen'
N[CAS=?c,NUM=sg,GEN=m] -> 'Luftballon'
N[CAS=?c,NUM=sg,GEN=m] -> 'Ball'
N[CAS=?c,NUM=sg,GEN=m] -> 'Ballon'
N[CAS=?c,NUM=sg,GEN=n] -> 'Tier'
N[CAS=?c,NUM=sg,GEN=n] -> 'Insekt'
N[CAS=?c,NUM=sg,GEN=f] -> 'Biene'
N[CAS=?c,NUM=sg,GEN=f] -> 'Weltkugel'
N[CAS=?c,NUM=sg,GEN=f] -> 'Erdkugel'
N[CAS=?c,NUM=sg,GEN=m] -> 'Globus'
N[CAS=?c,NUM=pl,GEN=m] -> 'Menschen'
N[CAS=?c,NUM=pl,GEN=f] -> 'Personen'
N[CAS=?c,NUM=pl,GEN=n] -> 'Kinder'
N[CAS=?c,NUM=pl,GEN=m] -> 'Streifen'
N[CAS=?c,NUM=sg,GEN=n] -> 'Flugobjekt'
N[CAS=?c,NUM=sg,GEN=n] -> 'Ufo'
N[CAS=dat,NUM=pl,GEN=m] -> 'Anzügen'
N[CAS=?c,NUM=sg,GEN=n] -> 'Spielzeug'
N[CAS=?c,NUM=sg,GEN=n] -> 'Lebewesen'
N[CAS=?c,NUM=sg,GEN=n] -> 'Wasser'
N[CAS=?c,NUM=sg,GEN=m] -> 'Fluss'
N[CAS=?c,NUM=sg,GEN=m] -> 'Roboter'
N[CAS=?c,NUM=sg,GEN=m] -> 'Rauch'
N[CAS=?c,NUM=sg,GEN=n] -> 'X'
N[CAS=?c,NUM=sg,GEN=n] -> 'Y'
N[CAS=?c,NUM=sg,GEN=n] -> 'Sigma'
N[CAS=?c,NUM=sg,GEN=n] -> 'Summenzeichen'
N[CAS=?c,NUM=sg,GEN=n] -> 'Gleichheitszeichen'
N[CAS=?c,NUM=sg,GEN=f] -> 'Summe'
N[CAS=?c,NUM=sg,GEN=f] -> 'Normalverteilung'
N[CAS=?c,NUM=sg,GEN=f] -> 'Gaußkurve'
N[CAS=?c,NUM=sg,GEN=m] -> 'Kolben'
N[CAS=dat,NUM=pl,GEN=m] -> 'Kugeln'
N[CAS=dat,NUM=pl,GEN=m] -> 'Kästen'
N[CAS=?c,NUM=sg,GEN=m] -> 'Ordner'
N[CAS=?c,NUM=sg,GEN=m] -> 'Kittel'
N[CAS=?c,NUM=sg,GEN=f] -> 'Brille'
N[CAS=?c,NUM=sg,GEN=f] -> 'Sprechblase'
N[CAS=dat,NUM=pl,GEN=m] -> 'Tasten'
N[CAS=dat,NUM=pl,GEN=m] -> 'Knöpfen'
N[CAS=?c,NUM=sg,GEN=f] -> 'Zwei'
N[CAS=?c,NUM=sg,GEN=f] -> 'Vier'
N[CAS=?c,NUM=sg,GEN=m] -> 'Punkt'
N[CAS=?c,NUM=sg,GEN=n] -> 'Komma'
N[CAS=?c,NUM=sg,GEN=f] -> 'Schrift'
N[CAS=?c,NUM=sg,GEN=f] -> 'Zeit'
N[CAS=dat,NUM=pl,GEN=m] -> 'Buchstaben'
N[CAS=?c,NUM=sg,GEN=n] -> 'A'
N[CAS=?c,NUM=sg,GEN=n] -> 'B'
N[CAS=dat,NUM=pl,GEN=m] -> 'Zeigern'
"""

In [6]:
ARTICLES = """
Det[CAS=nom,NUM=sg,GEN=m] -> 'der'
Det[CAS=gen,NUM=sg,GEN=m] -> 'des'
Det[CAS=dat,NUM=sg,GEN=m] -> 'dem'
Det[CAS=acc,NUM=sg,GEN=m] -> 'den'
Det[CAS=nom,NUM=sg,GEN=f] -> 'die'
Det[CAS=gen,NUM=sg,GEN=f] -> 'der'
Det[CAS=dat,NUM=sg,GEN=f] -> 'der'
Det[CAS=acc,NUM=sg,GEN=f] -> 'die'
Det[CAS=nom,NUM=sg,GEN=n] -> 'das'
Det[CAS=gen,NUM=sg,GEN=n] -> 'des'
Det[CAS=dat,NUM=sg,GEN=n] -> 'dem'
Det[CAS=acc,NUM=sg,GEN=n] -> 'das'
Det[CAS=nom,NUM=pl,GEN=?g] -> 'die'
Det[CAS=gen,NUM=pl,GEN=?g] -> 'der'
Det[CAS=dat,NUM=pl,GEN=?g] -> 'den'
Det[CAS=acc,NUM=pl,GEN=?g] -> 'die'
"""

In [7]:
BOS = """
Bos[CAS=nom] -> 'und jetzt'
Bos[CAS=acc] -> 'bitte zeig mir'
Bos[CAS=nom] -> 'das objekt ist'
Bos[CAS=nom] -> 'es ist'
Bos[CAS=acc] -> 'es gibt'
Bos[CAS=acc] -> 'zeig mal'
Bos[CAS=acc] -> 'wähl mal'
Bos[CAS=nom] -> 'das ding ist'
Bos[CAS=acc] -> 'ich nehme'
Bos[CAS=acc] -> 'bitte wähle'
Bos[CAS=acc] -> 'bitte finde'
Bos[CAS=nom] -> 'dann noch'
Bos[CAS=nom] -> 'dann'
Bos[CAS=acc] -> 'finde'
Bos[CAS=acc] -> 'wähle mal'
Bos[CAS=acc] -> 'ich sehe'
Bos[CAS=acc] -> 'nimm mal'
Bos[CAS=nom] -> 'der gegenstand ist'
Bos[CAS=acc] -> 'ich denke an'
Bos[CAS=acc] -> 'ich wähle'
Bos[CAS=acc] -> 'wähle'
Bos[CAS=acc] -> 'siehst du'
Bos[CAS=acc] -> 'zeig'
Bos[CAS=nom] -> 'als nächstes'
Bos[CAS=dat] -> 'weiter mit'
Bos[CAS=nom] -> 'bitte zeige'
Bos[CAS=nom] -> 'das nächste objekt ist'
Bos[CAS=acc] -> 'ich beschreibe'
Bos[CAS=nom] -> 'jetzt'
Bos[CAS=nom] -> 'der nächste gegenstand ist'
Bos[CAS=acc] -> 'du siehst'
Bos[CAS=acc] -> 'wähl'
Bos[CAS=acc] -> 'zeig mir'
Bos[CAS=acc] -> 'ich meine'
Bos[CAS=nom] -> 'das ist'
Bos[CAS=dat] -> 'weiter gehts mit'
Bos[CAS=acc] -> 'nimm'
Bos[CAS=acc] -> 'bitte nimm'
"""

In [8]:
ADJECTIVE_ENDINGS = """
AdjEnd[CAS=nom,NUM=sg,GEN=m] -> 'e'
AdjEnd[CAS=gen,NUM=sg,GEN=m] -> 'en'
AdjEnd[CAS=dat,NUM=sg,GEN=m] -> 'en'
AdjEnd[CAS=acc,NUM=sg,GEN=m] -> 'en'
AdjEnd[CAS=nom,NUM=sg,GEN=f] -> 'e'
AdjEnd[CAS=gen,NUM=sg,GEN=f] -> 'en'
AdjEnd[CAS=dat,NUM=sg,GEN=f] -> 'en'
AdjEnd[CAS=acc,NUM=sg,GEN=f] -> 'e'
AdjEnd[CAS=nom,NUM=sg,GEN=n] -> 'e'
AdjEnd[CAS=gen,NUM=sg,GEN=n] -> 'en'
AdjEnd[CAS=dat,NUM=sg,GEN=n] -> 'en'
AdjEnd[CAS=acc,NUM=sg,GEN=n] -> 'e'
AdjEnd[CAS=?c,NUM=pl,GEN=?g] -> 'en'
"""

In [9]:
NUM = {'sg', 'pl'}
CAS = {'nom', 'acc', 'gen', 'dat'}
GEN = {'m', 'f', 'n'}

def specify_rule(r):
    """specify feature grammar rule containing ? for all possible features and values"""
    new_rules = set([r])
    if '?g' in r:
        for g in GEN:
            new = set([rule.replace('?g', g) for rule in new_rules])
            new_rules = new_rules.union(new)  
    if '?c' in r:
        for c in CAS:
            new = set([rule.replace('?c', c) for rule in new_rules])
            new_rules = new_rules.union(new)
    if '?n' in r:
        for n in NUM:
            new = set([rule.replace('?n', n) for rule in new_rules])
            new_rules = new_rules.union(new)
    new_rules = new_rules.difference(set([rule for rule in new_rules if '?' in rule]))
    return list(new_rules)

In [10]:
def adjective_rules(a):
    """generate rules with inflection for an adjective"""
    adj_rules = []
    if a[-1].isdigit():  # no inflection for digits
        rule = f'Adj[CAS=?c,NUM=?n,GEN=?g] -> \'{a}\''
        for sr in specify_rule(rule):  # specify for all CNG combis
            adj_rules.append(sr)
    else:
        if a in {'lila', 'rosa'}:
            a += 'n'
        for rule in ADJECTIVE_ENDINGS.split('\n'):
            r = rule.replace('AdjEnd', 'Adj')
            r = r.replace('AdjEnd', 'Adj')
            r = r.replace('-> \'', f'-> \'{a}')
            for sr in specify_rule(r):  # specify for all CNG combis
                adj_rules.append(sr)
    return adj_rules

In [73]:
NONTERMINALS1 = """
S -> Bos[CAS=?c] NP[CAS=?c] Loc
NP[CAS=?c] -> Det[CAS=?c,NUM=?n,GEN=?g] Adj[CAS=?c,NUM=?n,GEN=?g] N[CAS=?c,NUM=?n,GEN=?g]"""

NONTERMINALS2 = """
S -> No Bos[CAS=?c] NP[CAS=?c] ContP Loc
S -> No Bos[CAS=?c] NP[CAS=?c] Loc NeighP
NP[CAS=?c] -> Det[CAS=?c,NUM=?n,GEN=?g] Adj[CAS=?c,NUM=?n,GEN=?g] N[CAS=?c,NUM=?n,GEN=?g]
ContP -> \'mit\' Det[CAS=dat,NUM=?n,GEN=?g] Cont[CAS=dat,NUM=?n,GEN=?g]
NeighP -> Nextto Det[CAS=dat,NUM=?n,GEN=?g] Neigh[CAS=dat,NUM=?n,GEN=?g]
Nextto -> \'neben\' | \'bei\' | \'in der Nähe von\'
No -> \'nein ,\' | \'nicht das ,\' | \'falsch ,\'"""

NONTERMINALS3 = """
S -> No Bos[CAS=?c] NP[CAS=?c] ContP Loc NeighP 
NP[CAS=?c] -> Det[CAS=?c,NUM=?n,GEN=?g] Adj[CAS=?c,NUM=?n,GEN=?g] N[CAS=?c,NUM=?n,GEN=?g]
ContP -> \'mit\' Det[CAS=dat,NUM=?n,GEN=?g] Cont[CAS=dat,NUM=?n,GEN=?g]
NeighP -> Nextto Det[CAS=dat,NUM=?n,GEN=?g] Neigh[CAS=dat,NUM=?n,GEN=?g]
Nextto -> \'neben\' | \'bei\' | \'in der Nähe von\'
No -> \'nein ,\' | \'nicht das ,\' | \'falsch ,\'"""

In [16]:
def CFG_transform(rule):
    rule = rule.replace('[', '_')
    rule = rule.replace(',', '_')
    rule = rule.replace('=', '')
    rule = rule.replace(']', '')
    rule = rule.replace('?', '')
    return rule

In [17]:
def split_rules(l, n=3):
    if len(l) > 2:
        split_len = len(l) // n
        return l[:split_len], l[split_len:(split_len*2)], l[(split_len*2):]
    else:
        if len(l) == 1:
            return l, l, l
        split_len = 1  # turn 2=0
        return l[:split_len], l[split_len:(split_len*2)], l[:split_len]

#### Merge slots, grammatical information to feature grammar

In [109]:
def featgram(i):
    """
    create a feature grammar for an item from
    - item-specific terminal rules
    - turn-specific non-terminal rules
    """
    d = items_dict[i]  # load item's dict
    rules = ["% start S"] + BOS.split('\n')  # initialize rules
    # specified article rules
    for r in ARTICLES.split('\n'):
        rules += specify_rule(r)
    
    # ITEM SPECIFIC terminal rules
    # item synonyms from slot
    synonyms = slots['object'][d["slot"].capitalize()]
    # check for object name and synonyms
    objs = slots['object'][d['slot'].capitalize()] + [i.capitalize()]
    obj_rules = [r for r in OBJECTS.split('\n') if '\'' in r and r.split('\'')[1] in objs]
    # location information
    locs = d['location']
    locs_transitive = {'neben', 'über', 'unter', 'hinter', 'auf', 'bei', 'vor'}
    loc_rules = [f'Loc -> \'{l}\'' for l in locs if l not in locs_transitive]  # ignore transitive location for now
    loc_rules1,loc_rules2,loc_rules3 = split_rules(loc_rules)
    # adjectives
    adj = d['adjectives']
    adj1, adj2, adj3 = split_rules(adj)
    
    # neighbors
    neighbors = d['neighbors']
    # content
    cont = d['content']
    
    # TURN 1
    rules1 = [] + rules
    # BOS + Det + Adj + Adj(ending) + hyperonym + location
    # e.g. wähle die blonde Person in der mitte
    # non terminal rules, replace general rule by specified rules
    for r in NONTERMINALS1.split('\n'):
        rules1 += specify_rule(r)
    hyps = d['hyperonyms']
    hyp_rules = [r for r in OBJECTS.split('\n') if '\'' in r and r.split('\'')[1].lower() in hyps]
    for r in hyp_rules:
        rules1 += specify_rule(r)
    for a in adj1:
        rules1 += adjective_rules(a)
    # BOS, articles and adjective endings
    rules1 += loc_rules1
    rules1 = [CFG_transform(r) for r in rules1 if r]
    fg1 = CFG.fromstring(rules1)
    #print(rules1)
    
    # TURN 2
    rules2 = [] + rules
    print(rules2)
    for r in NONTERMINALS2.split('\n'):
        rules2 += specify_rule(r)
    # check for object name and synonyms
    objs = slots['object'][d['slot'].capitalize()] + [i.capitalize()]
    obj_rules = [r for r in OBJECTS.split('\n') if '\'' in r and r.split('\'')[1] in objs]
    for r in obj_rules:
        rules2 += specify_rule(r)
    # content rules
    cont_rules = [r.replace('N[', 'Cont[') for r in OBJECTS.split('\n') if '\'' in r
                  and r.split('\'')[1].lower() in cont]
    for r in cont_rules:
        rules2 += specify_rule(r)
    # neighbor rules
    neigh_rules = [r.replace('N[', 'Neigh[') for r in OBJECTS.split('\n') if '\'' in r
                  and r.split('\'')[1].lower() in neighbors]
    for r in neigh_rules:
        rules2 += specify_rule(r)
    for a in adj2:
        rules2 += adjective_rules(a)
    rules2 += loc_rules2
    rules2 = [CFG_transform(r) for r in rules2 if r]
    fg2 = CFG.fromstring(rules2)
    print(rules2)
            
    # TURN 3
    rules3 = [] + rules
    for r in NONTERMINALS3.split('\n'):
        rules3 += specify_rule(r)
    # BOS + dict[adjectives] + Adj(ending) + slot + {bei/neben Det neighbors}
    objs = [r for r in OBJECTS.split('\n') if '\'' in r and r.split('\'')[1].lower() in d['slot']]
    for r in objs:
        rules3 += specify_rule(r)
    for r in neigh_rules:
        rules3 += specify_rule(r)
    for r in cont_rules:
        rules3 += specify_rule(r)
    rules3 += loc_rules3
    for a in adj3:
        rules3 += adjective_rules(a)
    rules3 = [CFG_transform(r) for r in rules3 if r]
    fg3 = CFG.fromstring(rules3)
    #print(rules3)
    # BOS, articles and adjective endings
#     rules += (BOS.split('\n') + loc_rules + adj_rules + art_rules)
#     rules = [CFG_transform(r) for r in rules if r]
#     fg = CFG.fromstring(rules)
    #fg = nltk.grammar.FeatureGrammar.fromstring('\n'.join(rules))
    #print(fg.productions())
    return fg1, fg2, fg3

In [110]:
g = featgram('diagramm')
print(len(g[0].productions()), len(g[1].productions()), len(g[2].productions()))
s01 = set(g[0].productions()).intersection(set(g[1].productions()))
s02 = set(g[0].productions()).intersection(set(g[2].productions()))
s12 = set(g[1].productions()).intersection(set(g[2].productions()))
len(s01), len(s02), len(s12)

['% start S', '', "Bos[CAS=nom] -> 'und jetzt'", "Bos[CAS=acc] -> 'bitte zeig mir'", "Bos[CAS=nom] -> 'das objekt ist'", "Bos[CAS=nom] -> 'es ist'", "Bos[CAS=acc] -> 'es gibt'", "Bos[CAS=acc] -> 'zeig mal'", "Bos[CAS=acc] -> 'wähl mal'", "Bos[CAS=nom] -> 'das ding ist'", "Bos[CAS=acc] -> 'ich nehme'", "Bos[CAS=acc] -> 'bitte wähle'", "Bos[CAS=acc] -> 'bitte finde'", "Bos[CAS=nom] -> 'dann noch'", "Bos[CAS=nom] -> 'dann'", "Bos[CAS=acc] -> 'finde'", "Bos[CAS=acc] -> 'wähle mal'", "Bos[CAS=acc] -> 'ich sehe'", "Bos[CAS=acc] -> 'nimm mal'", "Bos[CAS=nom] -> 'der gegenstand ist'", "Bos[CAS=acc] -> 'ich denke an'", "Bos[CAS=acc] -> 'ich wähle'", "Bos[CAS=acc] -> 'wähle'", "Bos[CAS=acc] -> 'siehst du'", "Bos[CAS=acc] -> 'zeig'", "Bos[CAS=nom] -> 'als nächstes'", "Bos[CAS=dat] -> 'weiter mit'", "Bos[CAS=nom] -> 'bitte zeige'", "Bos[CAS=nom] -> 'das nächste objekt ist'", "Bos[CAS=acc] -> 'ich beschreibe'", "Bos[CAS=nom] -> 'jetzt'", "Bos[CAS=nom] -> 'der nächste gegenstand ist'", "Bos[CAS=acc]

(86, 86, 144)

In [112]:
sents = set()
for sentence in generate(g[2], depth=10, n=10000):
    if len([j for i in sentence for j in i.split()]) >6:
        s = ' '.join(sentence).replace('_', ',')
        print(s)
        sents.add(s)

nein , weiter mit dem mathematischen Diagramm mit der Summe am rand neben der Uhr
nein , weiter mit dem mathematischen Diagramm mit der Summe am rand bei der Uhr
nein , weiter mit dem mathematischen Diagramm mit der Summe am rand in der Nähe von der Uhr
nein , weiter mit dem mathematischen Diagramm mit der Summe am rand neben dem Mikroskop
nein , weiter mit dem mathematischen Diagramm mit der Summe am rand bei dem Mikroskop
nein , weiter mit dem mathematischen Diagramm mit der Summe am rand in der Nähe von dem Mikroskop
nein , weiter mit dem mathematischen Diagramm mit der Summe an der wand neben der Uhr
nein , weiter mit dem mathematischen Diagramm mit der Summe an der wand bei der Uhr
nein , weiter mit dem mathematischen Diagramm mit der Summe an der wand in der Nähe von der Uhr
nein , weiter mit dem mathematischen Diagramm mit der Summe an der wand neben dem Mikroskop
nein , weiter mit dem mathematischen Diagramm mit der Summe an der wand bei dem Mikroskop
nein , weiter mit dem math

falsch , weiter mit dem mathematischen Diagramm mit dem Summenzeichen am rand bei der Uhr
falsch , weiter mit dem mathematischen Diagramm mit dem Summenzeichen am rand in der Nähe von der Uhr
falsch , weiter mit dem mathematischen Diagramm mit dem Summenzeichen am rand neben dem Mikroskop
falsch , weiter mit dem mathematischen Diagramm mit dem Summenzeichen am rand bei dem Mikroskop
falsch , weiter mit dem mathematischen Diagramm mit dem Summenzeichen am rand in der Nähe von dem Mikroskop
falsch , weiter mit dem mathematischen Diagramm mit dem Summenzeichen an der wand neben der Uhr
falsch , weiter mit dem mathematischen Diagramm mit dem Summenzeichen an der wand bei der Uhr
falsch , weiter mit dem mathematischen Diagramm mit dem Summenzeichen an der wand in der Nähe von der Uhr
falsch , weiter mit dem mathematischen Diagramm mit dem Summenzeichen an der wand neben dem Mikroskop
falsch , weiter mit dem mathematischen Diagramm mit dem Summenzeichen an der wand bei dem Mikroskop
falsch ,

nein , es gibt das mathematische A mit der Gaußkurve an der wand in der Nähe von der Uhr
nein , es gibt das mathematische A mit der Gaußkurve an der wand neben dem Mikroskop
nein , es gibt das mathematische A mit der Gaußkurve an der wand bei dem Mikroskop
nein , es gibt das mathematische A mit der Gaußkurve an der wand in der Nähe von dem Mikroskop
nein , es gibt das mathematische A mit dem X am rand neben der Uhr
nein , es gibt das mathematische A mit dem X am rand bei der Uhr
nein , es gibt das mathematische A mit dem X am rand in der Nähe von der Uhr
nein , es gibt das mathematische A mit dem X am rand neben dem Mikroskop
nein , es gibt das mathematische A mit dem X am rand bei dem Mikroskop
nein , es gibt das mathematische A mit dem X am rand in der Nähe von dem Mikroskop
nein , es gibt das mathematische A mit dem X an der wand neben der Uhr
nein , es gibt das mathematische A mit dem X an der wand bei der Uhr
nein , es gibt das mathematische A mit dem X an der wand in der Nähe von

nein , ich nehme das mathematische Diagramm mit dem Sigma am rand neben der Uhr
nein , ich nehme das mathematische Diagramm mit dem Sigma am rand bei der Uhr
nein , ich nehme das mathematische Diagramm mit dem Sigma am rand in der Nähe von der Uhr
nein , ich nehme das mathematische Diagramm mit dem Sigma am rand neben dem Mikroskop
nein , ich nehme das mathematische Diagramm mit dem Sigma am rand bei dem Mikroskop
nein , ich nehme das mathematische Diagramm mit dem Sigma am rand in der Nähe von dem Mikroskop
nein , ich nehme das mathematische Diagramm mit dem Sigma an der wand neben der Uhr
nein , ich nehme das mathematische Diagramm mit dem Sigma an der wand bei der Uhr
nein , ich nehme das mathematische Diagramm mit dem Sigma an der wand in der Nähe von der Uhr
nein , ich nehme das mathematische Diagramm mit dem Sigma an der wand neben dem Mikroskop
nein , ich nehme das mathematische Diagramm mit dem Sigma an der wand bei dem Mikroskop
nein , ich nehme das mathematische Diagramm mit 

nein , wähle mal das mathematische Diagramm mit dem Y am rand neben der Uhr
nein , wähle mal das mathematische Diagramm mit dem Y am rand bei der Uhr
nein , wähle mal das mathematische Diagramm mit dem Y am rand in der Nähe von der Uhr
nein , wähle mal das mathematische Diagramm mit dem Y am rand neben dem Mikroskop
nein , wähle mal das mathematische Diagramm mit dem Y am rand bei dem Mikroskop
nein , wähle mal das mathematische Diagramm mit dem Y am rand in der Nähe von dem Mikroskop
nein , wähle mal das mathematische Diagramm mit dem Y an der wand neben der Uhr
nein , wähle mal das mathematische Diagramm mit dem Y an der wand bei der Uhr
nein , wähle mal das mathematische Diagramm mit dem Y an der wand in der Nähe von der Uhr
nein , wähle mal das mathematische Diagramm mit dem Y an der wand neben dem Mikroskop
nein , wähle mal das mathematische Diagramm mit dem Y an der wand bei dem Mikroskop
nein , wähle mal das mathematische Diagramm mit dem Y an der wand in der Nähe von dem Mikros

...

nein , zeig mir das mathematische A mit dem Y an der wand in der Nähe von der Uhr
nein , zeig mir das mathematische A mit dem Y an der wand neben dem Mikroskop
nein , zeig mir das mathematische A mit dem Y an der wand bei dem Mikroskop
nein , zeig mir das mathematische A mit dem Y an der wand in der Nähe von dem Mikroskop
nein , zeig mir das mathematische A mit dem Sigma am rand neben der Uhr
nein , zeig mir das mathematische A mit dem Sigma am rand bei der Uhr
nein , zeig mir das mathematische A mit dem Sigma am rand in der Nähe von der Uhr
nein , zeig mir das mathematische A mit dem Sigma am rand neben dem Mikroskop
nein , zeig mir das mathematische A mit dem Sigma am rand bei dem Mikroskop
nein , zeig mir das mathematische A mit dem Sigma am rand in der Nähe von dem Mikroskop
nein , zeig mir das mathematische A mit dem Sigma an der wand neben der Uhr
nein , zeig mir das mathematische A mit dem Sigma an der wand bei der Uhr
nein , zeig mir das mathematische A mit dem Sigma an der wan

nein , bitte nimm das mathematische Diagramm mit der Normalverteilung an der wand neben dem Mikroskop
nein , bitte nimm das mathematische Diagramm mit der Normalverteilung an der wand bei dem Mikroskop
nein , bitte nimm das mathematische Diagramm mit der Normalverteilung an der wand in der Nähe von dem Mikroskop
nein , bitte nimm das mathematische Diagramm mit der Gaußkurve am rand neben der Uhr
nein , bitte nimm das mathematische Diagramm mit der Gaußkurve am rand bei der Uhr
nein , bitte nimm das mathematische Diagramm mit der Gaußkurve am rand in der Nähe von der Uhr
nein , bitte nimm das mathematische Diagramm mit der Gaußkurve am rand neben dem Mikroskop
nein , bitte nimm das mathematische Diagramm mit der Gaußkurve am rand bei dem Mikroskop
nein , bitte nimm das mathematische Diagramm mit der Gaußkurve am rand in der Nähe von dem Mikroskop
nein , bitte nimm das mathematische Diagramm mit der Gaußkurve an der wand neben der Uhr
nein , bitte nimm das mathematische Diagramm mit der 

nicht das , es gibt das mathematische A mit dem Sigma an der wand neben der Uhr
nicht das , es gibt das mathematische A mit dem Sigma an der wand bei der Uhr
nicht das , es gibt das mathematische A mit dem Sigma an der wand in der Nähe von der Uhr
nicht das , es gibt das mathematische A mit dem Sigma an der wand neben dem Mikroskop
nicht das , es gibt das mathematische A mit dem Sigma an der wand bei dem Mikroskop
nicht das , es gibt das mathematische A mit dem Sigma an der wand in der Nähe von dem Mikroskop
nicht das , es gibt das mathematische A mit dem Summenzeichen am rand neben der Uhr
nicht das , es gibt das mathematische A mit dem Summenzeichen am rand bei der Uhr
nicht das , es gibt das mathematische A mit dem Summenzeichen am rand in der Nähe von der Uhr
nicht das , es gibt das mathematische A mit dem Summenzeichen am rand neben dem Mikroskop
nicht das , es gibt das mathematische A mit dem Summenzeichen am rand bei dem Mikroskop
nicht das , es gibt das mathematische A mit dem 

nicht das , bitte wähle das mathematische Diagramm mit der Summe an der wand in der Nähe von der Uhr
nicht das , bitte wähle das mathematische Diagramm mit der Summe an der wand neben dem Mikroskop
nicht das , bitte wähle das mathematische Diagramm mit der Summe an der wand bei dem Mikroskop
nicht das , bitte wähle das mathematische Diagramm mit der Summe an der wand in der Nähe von dem Mikroskop
nicht das , bitte wähle das mathematische Diagramm mit der Normalverteilung am rand neben der Uhr
nicht das , bitte wähle das mathematische Diagramm mit der Normalverteilung am rand bei der Uhr
nicht das , bitte wähle das mathematische Diagramm mit der Normalverteilung am rand in der Nähe von der Uhr
nicht das , bitte wähle das mathematische Diagramm mit der Normalverteilung am rand neben dem Mikroskop
nicht das , bitte wähle das mathematische Diagramm mit der Normalverteilung am rand bei dem Mikroskop
nicht das , bitte wähle das mathematische Diagramm mit der Normalverteilung am rand in der N

nicht das , wähle mal das mathematische A mit dem Sigma am rand bei der Uhr
nicht das , wähle mal das mathematische A mit dem Sigma am rand in der Nähe von der Uhr
nicht das , wähle mal das mathematische A mit dem Sigma am rand neben dem Mikroskop
nicht das , wähle mal das mathematische A mit dem Sigma am rand bei dem Mikroskop
nicht das , wähle mal das mathematische A mit dem Sigma am rand in der Nähe von dem Mikroskop
nicht das , wähle mal das mathematische A mit dem Sigma an der wand neben der Uhr
nicht das , wähle mal das mathematische A mit dem Sigma an der wand bei der Uhr
nicht das , wähle mal das mathematische A mit dem Sigma an der wand in der Nähe von der Uhr
nicht das , wähle mal das mathematische A mit dem Sigma an der wand neben dem Mikroskop
nicht das , wähle mal das mathematische A mit dem Sigma an der wand bei dem Mikroskop
nicht das , wähle mal das mathematische A mit dem Sigma an der wand in der Nähe von dem Mikroskop
nicht das , wähle mal das mathematische A mit dem 

In [64]:
max100 = []
for sentence in generate(g[1], depth=10, n=10000):
    tokens = [j for i in sentence for j in i.split()]
    while len(max100) < 100:
        max100.append(tokens)
    shortest_sent = min(max100, key=lambda x: len(x))
    if len(tokens) >= len(shortest_sent):
        max100.append(tokens)
        max100.remove(shortest_sent)
print(max100)

[['nicht', 'das', 'das', 'ding', 'ist', 'das', 'schwarze', 'Objekt', 'mit', 'dem', 'X', 'rechts'], ['nicht', 'das', 'das', 'ding', 'ist', 'das', 'schwarze', 'Objekt', 'mit', 'dem', 'X', 'oben'], ['nicht', 'das', 'das', 'ding', 'ist', 'das', 'schwarze', 'Objekt', 'mit', 'dem', 'Y', 'rechts'], ['nicht', 'das', 'das', 'ding', 'ist', 'das', 'schwarze', 'Objekt', 'mit', 'dem', 'Y', 'oben'], ['nicht', 'das', 'das', 'ding', 'ist', 'das', 'schwarze', 'Objekt', 'mit', 'dem', 'Sigma', 'rechts'], ['nicht', 'das', 'das', 'ding', 'ist', 'das', 'schwarze', 'Objekt', 'mit', 'dem', 'Sigma', 'oben'], ['nicht', 'das', 'das', 'ding', 'ist', 'das', 'schwarze', 'Objekt', 'mit', 'dem', 'Summenzeichen', 'rechts'], ['nicht', 'das', 'das', 'ding', 'ist', 'das', 'schwarze', 'Objekt', 'mit', 'dem', 'Summenzeichen', 'oben'], ['nicht', 'das', 'das', 'ding', 'ist', 'das', 'schwarze', 'Ding', 'mit', 'der', 'Summe', 'rechts'], ['nicht', 'das', 'das', 'ding', 'ist', 'das', 'schwarze', 'Ding', 'mit', 'der', 'Summe', 'o

In [62]:
l = [[1,2], [], [1,2,3]]
min(l, key=lambda x: len(x))

[]

In [17]:
def run(turns=3, output=True):
    for i in items_dict.keys():
        utterance_dict = dict()
        for t in range(1,turns+1):  
            g = featgram(i, turn=1)
            sents = list()
            for sentence in generate(g, depth=5, n=10000):
                sents.append([j for i in sentence for j in i.split()])  # split multiword terminals
            utterance_dict[str(t-1)] = sents  #012
        if output:
            with open(f'utterances/{i}.json', 'w', encoding='utf-8') as f:
                json.dump(utterance_dict, f, indent=4, ensure_ascii=False)

In [18]:
run()

In [19]:
# TODOS
# turn 1: add adjectives, content; Loc Loc?
# remove/adjust das ist, der gegenstand ist
# check if all dict entries (nouns, neighbor, content) in OBJECTS
# Utterance Auswahl: set intersection mit vorherigen turns überprüfen? damit nicht immer selbes adjektiv
# anderes adjektiv je nach turn? 
# BOS turn 1+: only 'ich meine ...' or empty BOS?